<a href="https://colab.research.google.com/github/Tigi-koko/Favorita-Store-Forcasting/blob/main/Favorita_Corporaci%C3%B3n_Favorita_week3_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q xgboost joblib mlflow scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.6/752.6 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 13.1 MB/s eta 0:00:00


In [ ]:
# 1) Mount Drive and set directories
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
BASE_DIR = '/content/drive/MyDrive/corporacion_favorita'
DATA_DIR = os.path.join(BASE_DIR, 'data')
MODEL_DIR = os.path.join(BASE_DIR, 'model')
OUTPUT_DIR = os.path.join(BASE_DIR, 'results')
MLFLOW_DIR = os.path.join(BASE_DIR, 'mlruns')

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MLFLOW_DIR, exist_ok=True)

print("BASE_DIR:", BASE_DIR)
print("MLflow local store:", MLFLOW_DIR)

Mounted at /content/drive
BASE_DIR: /content/drive/MyDrive/corporacion_favorita
MLflow local store: /content/drive/MyDrive/corporacion_favorita/mlruns


In [ ]:
# 2) Imports
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import mlflow
import mlflow.sklearn
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
# 3) Load dataset (try common filenames)
possible_files = [
    os.path.join(DATA_DIR, 'data_clean.csv'),
    os.path.join(DATA_DIR, 'train.csv'),
    os.path.join(BASE_DIR, 'data_clean.csv'),
    os.path.join(BASE_DIR, 'train.csv')
]

df = None
for f in possible_files:
    if os.path.exists(f):
        print("Loading:", f)
        df = pd.read_csv(f, parse_dates=['date'], low_memory=False)
        break
if df is None:
    raise FileNotFoundError(f"No data file found. Checked: {possible_files}")

# Ensure 'unit_sales' exists
if 'unit_sales' not in df.columns:
    raise ValueError("CSV must contain 'unit_sales' column. Rename or provide correct file.")

Loading: /content/drive/MyDrive/corporacion_favorita/data_clean.csv


In [ ]:
# 4) Aggregate to daily total sales (simple baseline)
daily = df.groupby('date', as_index=False)['unit_sales'].sum().rename(columns={'unit_sales':'daily_sales'})
daily = daily.sort_values('date').reset_index(drop=True)
daily['date'] = pd.to_datetime(daily['date'])
daily = daily.set_index('date').asfreq('D')      # ensures contiguous dates
daily['daily_sales'] = daily['daily_sales'].fillna(0)

In [ ]:
# 5) Feature engineering (lags, rolling, date parts)
def create_features(series):
    df = series.to_frame(name='daily_sales').copy()
    df['date'] = df.index
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofweek'] = df.index.dayofweek
    df['is_weekend'] = df['dayofweek'].isin([5,6]).astype(int)
    df['weekofyear'] = df.index.isocalendar().week.astype(int)

    for lag in [1,7,14,28]:
        df[f'lag_{lag}'] = df['daily_sales'].shift(lag)
    df['roll_7_mean'] = df['daily_sales'].shift(1).rolling(7).mean()
    df['roll_14_mean'] = df['daily_sales'].shift(1).rolling(14).mean()
    df['roll_28_mean'] = df['daily_sales'].shift(1).rolling(28).mean()
    df = df.fillna(method='ffill').fillna(0)
    return df

df_feats = create_features(daily['daily_sales'])

/tmp/ipython-input-3870261902.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(0)


In [ ]:
# 6) Train/test split: use 2014-01-01..2014-03-31 as test if present otherwise last 90 days
start_2014 = pd.to_datetime('2014-01-01')
end_2014 = pd.to_datetime('2014-03-31')

if (df_feats.index.min() <= start_2014) and (df_feats.index.max() >= end_2014):
    train_df = df_feats[df_feats.index < start_2014]
    test_df = df_feats[(df_feats.index >= start_2014) & (df_feats.index <= end_2014)]
    print("Using 2014-01-01..2014-03-31 as test set.")
else:
    test_size = 90
    train_df = df_feats.iloc[:-test_size]
    test_df = df_feats.iloc[-test_size:]
    print(f"Using last {test_size} days as test set.")

TARGET = 'daily_sales'
FEATURES = [c for c in df_feats.columns if c not in [TARGET, 'date']]

X_train = train_df[FEATURES].values
y_train = train_df[TARGET].values
X_test = test_df[FEATURES].values
y_test = test_df[TARGET].values

Using 2014-01-01..2014-03-31 as test set.


In [ ]:
# 7) Scale
scaler = StandardScaler()
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)
scaler_path = os.path.join(MODEL_DIR, 'scaler.pkl')
joblib.dump(scaler, scaler_path)
print("Saved scaler:", scaler_path)

# 8) Metric functions (adds Bias, MAD, rMAD, MAPE)
def bias(y_true, y_pred):
    return np.mean(y_pred - y_true)

def mad(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmad(y_true, y_pred):
    denom = np.mean(np.abs(y_true))
    if denom == 0:
        return np.nan
    return np.mean(np.abs(y_pred - y_true)) / denom

def mape(y_true, y_pred):
    with np.errstate(divide='ignore', invalid='ignore'):
        res = np.abs((y_true - y_pred) / np.where(y_true==0, np.nan, y_true))
    return np.nanmean(res) * 100

def evaluate_all(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    b = bias(y_true, y_pred)
    _mad = mad(y_true, y_pred)
    _rmad = rmad(y_true, y_pred)
    _mape = mape(y_true, y_pred)
    return {'RMSE': rmse, 'MAE': mae, 'Bias': b, 'MAD': _mad, 'rMAD': _rmad, 'MAPE': _mape}


Saved scaler: /content/drive/MyDrive/corporacion_favorita/model/scaler.pkl


In [ ]:
# 9) Setup MLflow to use file store inside your Drive
mlflow.set_tracking_uri(f"file://{MLFLOW_DIR}")
experiment_name = "favorita-week3-xgb"
mlflow.set_experiment(experiment_name)
print("MLflow tracking URI:", mlflow.get_tracking_uri())
print("Experiment name:", experiment_name)

MLflow tracking URI: file:///content/drive/MyDrive/corporacion_favorita/mlruns
Experiment name: favorita-week3-xgb


In [ ]:
# 10) Baseline model: load if exists else train a simple XGB with baseline params
baseline_path = os.path.join(MODEL_DIR, 'xgb_baseline.pkl')
baseline_params = dict(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42, n_jobs= -1)

if os.path.exists(baseline_path):
    print("Loading existing baseline model:", baseline_path)
    baseline_model = joblib.load(baseline_path)
else:
    print("Training baseline XGBoost...")
    baseline_model = xgb.XGBRegressor(**baseline_params)
    baseline_model.fit(X_train_s, y_train)
    joblib.dump(baseline_model, baseline_path)
    print("Saved baseline model:", baseline_path)

Loading existing baseline model: /content/drive/MyDrive/corporacion_favorita/model/xgb_baseline.pkl


In [ ]:
# 11) Baseline evaluation
y_pred_baseline = baseline_model.predict(X_test_s)
metrics_baseline = evaluate_all(y_test, y_pred_baseline)
print("Baseline metrics:", metrics_baseline)

# Save plot (actual vs predicted) to artifact and local folder
plt.figure(figsize=(10,4))
plt.plot(test_df.index, y_test, label='actual')
plt.plot(test_df.index, y_pred_baseline, label='predicted')
plt.title('Baseline: Actual vs Predicted')
plt.legend()
baseline_plot_path = os.path.join(OUTPUT_DIR, 'baseline_actual_vs_pred.png')
plt.savefig(baseline_plot_path, bbox_inches='tight')
plt.close()


Baseline metrics: {'RMSE': np.float64(21.764486769986913), 'MAE': 15.611165746053059, 'Bias': np.float64(3.5465384165445966), 'MAD': np.float64(15.611165746053059), 'rMAD': np.float64(0.31424847173893433), 'MAPE': np.float64(33.4146603522543)}


In [ ]:
# 12) Log baseline run to MLflow (params, metrics, artifact)
with mlflow.start_run(run_name='baseline_xgb'):
    mlflow.log_params({'model':'xgboost_baseline', **baseline_params})
    mlflow.log_metrics(metrics_baseline)
    mlflow.sklearn.log_model(baseline_model, artifact_path='model_baseline')
    mlflow.log_artifact(baseline_plot_path, artifact_path='plots')
    # log feature list and scaler as artifacts
    features_path = os.path.join(OUTPUT_DIR, 'features_baseline.txt')
    with open(features_path, 'w') as f:
        f.write('\n'.join(FEATURES))
    mlflow.log_artifact(features_path, artifact_path='meta')
    mlflow.log_artifact(scaler_path, artifact_path='meta')
print("Baseline run logged to MLflow.")

2025/10/28 09:02:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 09:02:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Baseline run logged to MLflow.


In [ ]:
# 13) Hyperparameter tuning with RandomizedSearchCV (time-series aware with TimeSeriesSplit)
# Keep search small to run quickly on Colab
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 1.0],
    'reg_lambda': [1.0, 2.0]
}

xgb_est = xgb.XGBRegressor(random_state=42, n_jobs=-1)
tscv = TimeSeriesSplit(n_splits=3)
search = RandomizedSearchCV(
    estimator=xgb_est,
    param_distributions=param_dist,
    n_iter=20,              # keep moderate; reduce to 8-10 for faster runs
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    verbose=1,
    random_state=42,
    n_jobs=1
)

print("Starting RandomizedSearchCV (this can take several minutes)...")
search.fit(X_train_s, y_train)
print("Best params from search:", search.best_params_)
best_params = search.best_params_

Starting RandomizedSearchCV (this can take several minutes)...
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best params from search: {'subsample': 0.9, 'reg_lambda': 1.0, 'reg_alpha': 0, 'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.03, 'colsample_bytree': 1.0}


In [ ]:
# 14) Retrain best model on full training set
best_model = xgb.XGBRegressor(**best_params, random_state=42, n_jobs=-1)
best_model.fit(X_train_s, y_train)
best_path = os.path.join(MODEL_DIR, 'xgb_tuned.pkl')
joblib.dump(best_model, best_path)
print("Saved tuned model:", best_path)

Saved tuned model: /content/drive/MyDrive/corporacion_favorita/model/xgb_tuned.pkl


In [ ]:
# 15) Evaluate tuned model on test set
y_pred_best = best_model.predict(X_test_s)
metrics_best = evaluate_all(y_test, y_pred_best)
print("Tuned metrics:", metrics_best)
# Save tuned plot
plt.figure(figsize=(10,4))
plt.plot(test_df.index, y_test, label='actual')
plt.plot(test_df.index, y_pred_best, label='predicted_tuned')
plt.title('Tuned XGB: Actual vs Predicted')
plt.legend()
tuned_plot_path = os.path.join(OUTPUT_DIR, 'tuned_actual_vs_pred.png')
plt.savefig(tuned_plot_path, bbox_inches='tight')
plt.close()

Tuned metrics: {'RMSE': np.float64(24.51409477472981), 'MAE': 16.36393199496799, 'Bias': np.float64(4.80888311598036), 'MAD': np.float64(16.36393199496799), 'rMAD': np.float64(0.3294014492388994), 'MAPE': np.float64(35.188322013284186)}


In [ ]:
# 16) Log tuned run to MLflow
with mlflow.start_run(run_name='tuned_xgb'):
    mlflow.log_params({'model':'xgboost_tuned', **best_params})
    mlflow.log_metrics(metrics_best)
    mlflow.sklearn.log_model(best_model, artifact_path='model_tuned')
    mlflow.log_artifact(tuned_plot_path, artifact_path='plots')
    # log artifacts: features, scaler
    mlflow.log_artifact(features_path, artifact_path='meta')
    mlflow.log_artifact(scaler_path, artifact_path='meta')
print("Tuned run logged to MLflow.")


2025/10/28 09:02:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/28 09:02:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Tuned run logged to MLflow.


In [ ]:
# 17) Save feature list and pipeline metadata for next week
features_file = os.path.join(MODEL_DIR, 'features.txt')
with open(features_file, 'w') as f:
    f.write('\n'.join(FEATURES))

pipeline_meta = {
    'model_baseline': baseline_path,
    'model_tuned': best_path,
    'scaler': scaler_path,
    'features': features_file,
    'trained_until': str(train_df.index.max())
}
joblib.dump(pipeline_meta, os.path.join(MODEL_DIR, 'pipeline_meta.pkl'))
print("Saved pipeline metadata:", os.path.join(MODEL_DIR, 'pipeline_meta.pkl'))

Saved pipeline metadata: /content/drive/MyDrive/corporacion_favorita/model/pipeline_meta.pkl


In [ ]:
# 18) Print helpful pointers and MLflow UI instructions
print("\n=== DONE ===")
print("Baseline metrics:", metrics_baseline)
print("Tuned metrics:", metrics_best)
print("Saved models to:", baseline_path, best_path)
print("Saved artifacts (plots, features) to:", OUTPUT_DIR, MODEL_DIR)

print("\nTo view MLflow UI locally from Colab, run this command in a new cell (it will run a small server):")
print("!mlflow ui --backend-store-uri file://{mlruns} --port 5000".format(mlruns=MLFLOW_DIR))
print("Then open the forwarded port (or use ngrok) OR download the mlruns folder and view with MLflow locally.")
print("You can also explore the mlruns folder in Drive: ", MLFLOW_DIR)



=== DONE ===
Baseline metrics: {'RMSE': np.float64(21.764486769986913), 'MAE': 15.611165746053059, 'Bias': np.float64(3.5465384165445966), 'MAD': np.float64(15.611165746053059), 'rMAD': np.float64(0.31424847173893433), 'MAPE': np.float64(33.4146603522543)}
Tuned metrics: {'RMSE': np.float64(24.51409477472981), 'MAE': 16.36393199496799, 'Bias': np.float64(4.80888311598036), 'MAD': np.float64(16.36393199496799), 'rMAD': np.float64(0.3294014492388994), 'MAPE': np.float64(35.188322013284186)}
Saved models to: /content/drive/MyDrive/corporacion_favorita/model/xgb_baseline.pkl /content/drive/MyDrive/corporacion_favorita/model/xgb_tuned.pkl
Saved artifacts (plots, features) to: /content/drive/MyDrive/corporacion_favorita/results /content/drive/MyDrive/corporacion_favorita/model

To view MLflow UI locally from Colab, run this command in a new cell (it will run a small server):
!mlflow ui --backend-store-uri file:///content/drive/MyDrive/corporacion_favorita/mlruns --port 5000
Then open the fo

In [22]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load your training data (replace with your actual dataset)
# Using the same file used for initial data loading in cell CSUV0tRNP3qA
train_df = pd.read_csv('/content/drive/MyDrive/corporacion_favorita/data_clean.csv', parse_dates=['date'], low_memory=False)

# Use the same feature list used before
# Corrected the file path to match where it was saved in cell 0CgmlyUdTzp-
features_file_path = '/content/drive/MyDrive/corporacion_favorita/model/features.txt'
with open(features_file_path, 'r') as f:
    features = f.read().splitlines()

# Fit the scaler again
scaler = StandardScaler()
# Before fitting, ensure that the features in train_df are the same used for training
# In this notebook, the features were created after aggregating to daily sales and feature engineering
# The train_df used here is the original data_clean.csv, which doesn't have the engineered features
# To correctly scale, we need to apply the same feature engineering steps to the loaded data
# and then select the features used for training.

# Re-applying the feature engineering function from cell pywODXn8QtZG
def create_features(series):
    df = series.to_frame(name='daily_sales').copy()
    df['date'] = df.index
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofweek'] = df.index.dayofweek
    df['is_weekend'] = df['dayofweek'].isin([5,6]).astype(int)
    df['weekofyear'] = df.index.isocalendar().week.astype(int)

    for lag in [1,7,14,28]:
        df[f'lag_{lag}'] = df['daily_sales'].shift(lag)
    df['roll_7_mean'] = df['daily_sales'].shift(1).rolling(7).mean()
    df['roll_14_mean'] = df['daily_sales'].shift(1).rolling(14).mean()
    df['roll_28_mean'] = df['daily_sales'].shift(1).rolling(28).mean()
    # Using .ffill() and .fillna(0) as in the original feature engineering cell
    df = df.ffill().fillna(0)
    return df

# Aggregate the loaded data to daily total sales as done in cell USMZllRlQUiv
daily_for_scaling = train_df.groupby('date', as_index=False)['unit_sales'].sum().rename(columns={'unit_sales':'daily_sales'})
daily_for_scaling = daily_for_scaling.sort_values('date').reset_index(drop=True)
daily_for_scaling['date'] = pd.to_datetime(daily_for_scaling['date'])
daily_for_scaling = daily_for_scaling.set_index('date').asfreq('D')      # ensures contiguous dates
daily_for_scaling['daily_sales'] = daily_for_scaling['daily_sales'].fillna(0)

# Create features for scaling using the function
df_feats_for_scaling = create_features(daily_for_scaling['daily_sales'])

# Select only the features used for training
X_for_scaling = df_feats_for_scaling[features]


scaler.fit(X_for_scaling)

# Save the scaler again properly
# Using the correct path as in cell Nm11q1hGQ6ET and 0CgmlyUdTzp-
scaler_path = '/content/drive/MyDrive/corporacion_favorita/model/scaler.pkl'
import joblib # Changed to joblib for consistency with other cell
joblib.dump(scaler, scaler_path)


print("✅ Scaler recreated and saved successfully!")

✅ Scaler recreated and saved successfully!


In [24]:
import joblib # Import joblib
with open('/content/drive/MyDrive/corporacion_favorita/model/scaler.pkl', 'rb') as f:
    scaler = joblib.load(f) # Use joblib.load
print("Scaler now loads fine!")

Scaler now loads fine!


In [25]:
import sklearn
print(sklearn.__version__)


1.6.1


In [26]:
!pip install scikit-learn==1.2.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 49.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-learn: filename=scikit_learn-1.2.2-cp312-cp312-linux_x86_64.whl size=9847599 sha256=96c19a1f5abd65299079bd92ea86b4647c7b12bba48d0e3960a631893c61a85c
  Stored in directory: /root/.cache/pip/wheels/24/f8/77/ae90c181b806f450a6fec8c8f794594e7c92fa79d7ca27e656
Successfully built scikit-learn
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml-cu12 25.6.0 requires scikit-learn>=1.5, but you have scikit-learn 1.2.2 which is incompatible.
imbalanced-learn 0.1